# BertTopic Demo 
From the result, the model we found for suicidal text that was most optimal was the BerTopic model

Download stuff (if u dun have them)

In [ ]:
!pip install bertopic
!pip install symspellpy
!pip install pandas==1.3.5
!pip install joblib==1.2.0


In [17]:
!pip install numpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load libs & files

In [1]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

import bertopic
import pandas as pd
from bertopic import BERTopic
import joblib

#read from gdrive
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/TxtAnalytics_Proj



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Mounted at /content/gdrive
/content/gdrive/MyDrive/TxtAnalytics_Proj


In [16]:

#print('The pickle version is {}.'.format(pickle.__version__))
print('The bertopic version is {}.'.format(np.__version__))
topic_model = BERTopic.load('bertTopic_suicidal_model.jl')
Probs = joblib.load('bertTopic_suicidal_probs.jl')
topics = joblib.load('bertTopic_suicidal_topic.jl')


The bertopic version is 1.22.4.


KeyError: ignored

In [ ]:
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re
import gensim
import symspellpy
from symspellpy import SymSpell, Verbosity
import pkg_resources

def get_wordnet_pos(treebank_tag):
  """
  return WORDNET POS compliance to WORDNET lemmatization (a,n,r,v) 
  """
  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
  # As default pos in lemmatization is Noun
    return wordnet.NOUN

def pos_tag_and_lemmatize(sentence):
# find the pos tagging for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....

  pos_tokens = [nltk.pos_tag(word_tokenize(sentence))]
  lemmatizer = WordNetLemmatizer()
  # lemmatization using pos tag  
  pos_tokens = [ [(token, lemmatizer.lemmatize(token, get_wordnet_pos(pos_tag)), [pos_tag]) for (token,pos_tag) in pos] for pos in pos_tokens]
  return pos_tokens

def getLemmatizedText(pos_tag):
  lst = list()
  for postag_tuple in pos_tag:
    for p in postag_tuple:
      value = re.sub(r'[^a-zA-Z ]', '', p[1])
      lst.append(value)

  return lst

def preprocess(text):
  #to lower case
  result = text.str.lower()

  #remove punctuation and numbers
  result = result.apply(lambda t: re.sub(r'[^a-zA-Z ]', '', t))

  result = result.apply(lambda t: gensim.parsing.preprocessing.remove_stopwords(str(t)))

  symsp = SymSpell(max_dictionary_edit_distance=1, prefix_length=7) #Damerau-Levenshtein algorithm = 2, Levenshtein = 1
  dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
  symsp.load_dictionary(dictionary_path, term_index=0, count_index=1)
  result = result.apply(lambda t: symsp.lookup_compound(t, max_edit_distance=1)[0].term )
  result = result.apply(lambda t: gensim.parsing.preprocessing.remove_stopwords(str(t)))

   #lemmatization, note: Lemmatization includes stemming as discussed in class.
  result_pos_tag = result.apply(lambda t: pos_tag_and_lemmatize(t))

  result_lemmatized = result_pos_tag.apply(lambda t: getLemmatizedText(t))

  return result_lemmatized


In [ ]:
def predict_topic(model, vectorizer, text):

    if len(text) == 0:
      return None, -1
    else:
      processedText = preprocess(text)
     
      topics, topic_probability_scores = model.transform(processedText)
      top_topics = np.argmax(topic_probability_scores, axis=1)

      # Map topic IDs to topic labels
      topic_labels = model.get_topic_freq().Topic.to_dict()

      # Get the topic labels for each document
      topic_labels = [topic_labels.get(topic_id) for topic_id in top_topics]

      # Print the topic labels for each document
      infer_topicNo = topic_labels[0]
      topic_words = model.get_topic(topic=infer_topicNo)
      word_lists = []
      for w in topic_words:
          word_lists.append(w[0])

      topic = word_lists
      #print (topic)
      #for i, text in enumerate(text):
      #    print(f"Document {i}: nTopic: {topic_labels[i]}")
      return topic, infer_topicNo

Usage

In [ ]:
mytext = [""" i wanna jump from an elevated place. i wanna kill myself, like really really would like to jump off an elevated place, but not die what do i do? drugs? fall from somewhere not far above the ground(like a 4meters jump)? you know, i'd just like to die but i have a gr8 life other than that (I have different lives in two different countries, just want to kill myself in one)

         """]

topic, infer_topicNo = predict_topic(topic_model, vectorizer = topic_model.vectorizer_model, text = mytext)

print("Topic: " + str(infer_topicNo) )
print(topic)